## Consultando as tabelas criadas

Nesta seção, vamos aprender como consultar as tabelas que foram criadas em nosso ambiente Databricks. Entender como acessar e visualizar os dados armazenados é fundamental para realizar análises, validar resultados e dar continuidade ao processamento de dados.

In [0]:
%sql
select * from bike_store.clients;

In [0]:
%sql
--  Consulta de Contatos de Clientes
select 
  customer_id
  ,first_name
  ,phone
  ,email
from bike_store.clients; 

In [0]:
%sql
--  Consulta de Contatos de Clientes tem contém sha no primeiro nome
select 
    customer_id
    ,first_name
    ,phone
    ,email
from bike_store.clients
where first_name like "%sha%";


## Transformando consulta SQL em dataframes

Transformar consultas SQL em DataFrames é uma prática comum no Databricks, permitindo combinar a flexibilidade da linguagem SQL com o poder de processamento dos DataFrames do Spark. Dessa forma, é possível realizar análises avançadas, manipular dados de maneira eficiente e integrar diferentes etapas do pipeline de dados utilizando tanto comandos SQL quanto operações em DataFrames.

In [0]:
df =  spark.sql('''
    select
        customer_id as id
        ,first_name as name
        ,phone as phone
        ,email as email
    from bike_store.clients
    where first_name like "%sha%";         
'''
)

In [0]:
display(df)

In [0]:
display(spark.sql("select * from bike_store.clients"))

## Salvando o dataframe em diferentes formatos

Salvar DataFrames em diferentes formatos é uma etapa essencial no processamento de dados, pois permite compartilhar, armazenar e reutilizar informações de maneira eficiente. No Databricks, é possível exportar DataFrames para diversos formatos populares, como CSV, Parquet, Delta e outros, facilitando a integração com diferentes sistemas e fluxos de trabalho. Nesta seção, veremos como realizar esse procedimento de forma prática.

In [0]:
df =  spark.sql('''
    select 
        customer_id as id
        ,first_name as name
        ,phone as phone
        ,email as email
    from bike_store.clients
    where first_name like "%sha%"
''')

In [0]:
display(df)

In [0]:
dbutils.fs.ls('/Volumes/workspace/bike_store/outputs')

### Salvando no formato Parquet

O formato Parquet é amplamente utilizado em ambientes de Big Data por ser eficiente no armazenamento e leitura de grandes volumes de dados. Ele organiza as informações de forma colunar, o que permite compressão otimizada e consultas mais rápidas, especialmente em operações analíticas. Nesta seção, veremos como salvar DataFrames no formato Parquet utilizando o Databricks, garantindo portabilidade e performance no processamento dos dados.

#### Termos Tecnicos Para voce ir treinando 
* **write**:  Escrever/ Gravar

**Modos de Escritas Mais usados**:

* **overwrite**   = se existe ele sobescreve/substitui
* **append**      = Mantem o existente e adiciona o conteudo no final 
* **ignore**      = Usado para salvar os dados apenas se o local de destino não existir



In [0]:
df.write.mode("overwrite").parquet("/Volumes/workspace/bike_store/outputs/contact_sha.parquet")

In [0]:
display(dbutils.fs.ls('/Volumes/workspace/bike_store/outputs'))

In [0]:
display(dbutils.fs.ls('/Volumes/workspace/bike_store/outputs/contact_sha.parquet'))

### Salvar em CSV e JSON

Salvar DataFrames em diferentes formatos, como CSV e JSON, é uma prática fundamental para garantir a portabilidade e o compartilhamento dos dados processados. Esses formatos são amplamente utilizados em integrações com outros sistemas, análises externas e armazenamento de dados intermediários. A seguir, veremos como exportar DataFrames para CSV e JSON no Databricks de forma simples e eficiente.

In [0]:
# Salvar em CSV e JSON
df.write \
    .option("delimiter", ",") \
    .mode("overwrite") \
    .csv("/Volumes/workspace/bike_store/outputs/contact_sha.csv")


In [0]:
df.write \
    .mode("overwrite") \
    .json("/Volumes/workspace/bike_store/outputs/contact_sha.json")

In [0]:
display(dbutils.fs.ls('/Volumes/workspace/bike_store/outputs'))

## Ler arquivos CSV , Json e Parquet

Ler arquivos em diferentes formatos é uma etapa essencial no processamento de dados, pois permite integrar informações provenientes de diversas fontes e sistemas. No Databricks, é possível importar dados em formatos como CSV, JSON e Parquet de maneira simples e eficiente, facilitando a análise e o tratamento dos dados. Nesta seção, veremos como realizar a leitura desses arquivos e transformar seu conteúdo em DataFrames para posterior manipulação.

In [0]:
path = "/Volumes/workspace/bike_store/outputs"

In [0]:
# Ler o arquivo CSV em um DataFrame
df_csv = spark.read.csv(f"{path}/contact_sha.csv", header=False, inferSchema=True)
display(df_csv)



In [0]:
# Ler o arquivo JSON em um DataFrame
df_json = spark.read.json(f"{path}/contact_sha.json")
display(df_json)

In [0]:
# Ler o arquivo Sem Salvar em Um DF
df_parquet =  spark.read.parquet(f"{path}/contact_sha.parquet")
display(df_parquet)

## Gravação e Leitura particionado

A gravação e leitura de dados particionados é uma técnica fundamental para otimizar o desempenho e a organização dos dados em ambientes de Big Data, como o Databricks. Ao particionar arquivos, é possível dividir grandes volumes de dados em subconjuntos menores com base em valores de uma ou mais colunas, facilitando consultas mais rápidas e eficientes. Nesta seção, veremos como salvar e ler arquivos particionados, destacando as vantagens dessa abordagem para o processamento e análise de dados.

In [0]:
%sql
select * from bike_store.clients;

In [0]:
novodf = spark.sql('select * from bike_store.clients')
display(novodf)

In [0]:
# Gravar Particionado
novodf.write\
    .partitionBy("state") \
    .mode("overwrite") \
    .parquet('/Volumes/workspace/bike_store/outputs/contact_sha.parquet')

In [0]:
display(dbutils.fs.ls('/Volumes/workspace/bike_store/outputs/contact_sha.parquet/state=CA/'))


In [0]:
# Ler Arquivo particionado 
state_CA = spark.read.parquet('/Volumes/workspace/bike_store/outputs/contact_sha.parquet/state=CA/')
display(state_CA)

In [0]:
display(dbutils.fs.ls('dbfs:/FileStore/tables/DiferentesSaidas/Contato_Particionado.parquet'))

In [0]:
#Ler Arquivo particionado (Geral) 
state_CA = spark.read.parquet('dbfs:/FileStore/tables/DiferentesSaidas/Contato_Particionado.parquet')
display(state_CA)